In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

2024-06-18 23:16:57.233466: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 23:16:57.444206: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 23:16:57.444249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 23:16:57.489989: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 23:16:57.578077: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 23:16:57.579525: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
sample_rate = 16000

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-06-18 23:17:01.609855: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 23:17:01.610255: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-18 23:17:01.725276: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-18 23:17:01.742910: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

In [8]:
BASE_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_baseline.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48000, 1)]           0         []                            
                                                                                                  
 conv1 (Conv1D)              (None, 24000, 28)            224       ['input_1[0][0]']             
                                                                                                  
 maxpool1 (MaxPooling1D)     (None, 11999, 28)            0         ['conv1[0][0]']               
                                                                                                  
 fire2_squeeze (Conv1D)      (None, 11999, 4)             116       ['maxpool1[0][0]']            
                                                                                              

### Convert to TF Lite format

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpfsdj4dt6/assets


INFO:tensorflow:Assets written to: /tmp/tmpfsdj4dt6/assets
2024-06-18 23:17:56.306069: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 23:17:56.306120: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 23:17:56.307217: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpfsdj4dt6
2024-06-18 23:17:56.313174: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 23:17:56.313194: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpfsdj4dt6
2024-06-18 23:17:56.326297: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-18 23:17:56.332015: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 23:17:56.522972: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [10]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Basic assessment of the whole dataset (without any partitions):
Accuracy: 90.14%
Recall: 97.88%
Precision: 78.57%
F1-score: 87.17%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.8725800678677726

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.8693880787159165

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.819805995108022

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.8143614533123427

Time for Test dataset:
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%

Time to make a prediction for a single data point
Mean: 0.012 seconds
Max: 0.027 seconds
Min: 0.01 seconds


M

In [11]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 150.871 Kilobytes


### Dynamic range quantization

In [12]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp5_5luwic/assets


INFO:tensorflow:Assets written to: /tmp/tmp5_5luwic/assets
2024-06-18 23:22:29.435284: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 23:22:29.435340: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 23:22:29.435502: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp5_5luwic
2024-06-18 23:22:29.441516: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 23:22:29.441545: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp5_5luwic
2024-06-18 23:22:29.458342: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 23:22:29.641138: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp5_5luwic
2024-06-18 23:22:29.696425: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [13]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 90.22%
Recall: 97.88%
Precision: 78.71%
F1-score: 87.25%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.8734148572290618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.8681081934413141

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.819805995108022

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.819507320857399

Time for Test dataset:
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%

Time to make a prediction for a single data point
Mean: 0.02 seconds
Max: 0.056 seconds
Min

In [14]:
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

File size: 89.656 Kilobytes


### Float16 quantization

In [15]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpc3js1y3m/assets


INFO:tensorflow:Assets written to: /tmp/tmpc3js1y3m/assets
2024-06-18 23:30:00.914360: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 23:30:00.914410: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 23:30:00.914610: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpc3js1y3m
2024-06-18 23:30:00.921435: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 23:30:00.921460: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpc3js1y3m
2024-06-18 23:30:00.940701: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 23:30:01.139443: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpc3js1y3m
2024-06-18 23:30:01.201815: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [16]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 90.14%
Recall: 97.88%
Precision: 78.57%
F1-score: 87.17%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.8725800678677726

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.8696262996033635

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.819805995108022

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.8193236300708324

Time for Test dataset:
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%

Time to make a prediction for a single data point
Mean: 0.013 seconds
Max: 0.026 seconds
M

In [17]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 97.117 Kilobytes


### Full integer quantization

In [18]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_test_np).batch(1).take(500):
    # Model has only one input so each data point has one element.
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

full_int_quant_model = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpwrdbqxoq/assets


INFO:tensorflow:Assets written to: /tmp/tmpwrdbqxoq/assets
/home/polina/test/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-18 23:34:21.436162: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 23:34:21.436216: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 23:34:21.436382: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpwrdbqxoq
2024-06-18 23:34:21.443027: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 23:34:21.443050: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpwrdbqxoq
2024-06-18 23:34:21.460264: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 23:34:21.

In [19]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(full_int_quant_model, x_val_np, y_val_np, model_format="tf_lite", input_data_uint8_type=True)

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(full_int_quant_model, x_test_np, y_test_np, model_format="tf_lite", input_data_uint8_type=True)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(full_int_quant_model, x_test_np, y_test_np, model_format="tf_lite", input_data_uint8_type=True)

FULL_INT_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_full_int_q.tflite"
open(FULL_INT_QUANT_MODEL_FILE_NAME, "wb").write(full_int_quant_model)
print("\n")
print("Model file name: ", FULL_INT_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FULL_INT_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 65.80%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Non-overlap mean F1-score:  0.0

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.0

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Non-overlap mean F1-score:  0.0

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.0

Time for Test dataset:
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Time to make a prediction for a single data point
Mean: 0.015 seconds
Max: 0.031 seconds
Min: 0.014 seconds


Model file name:  time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_full_int_q.tflite
File size: 0.085 Megabytes


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Float fallback

In [20]:
float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpybb6rhno/assets


INFO:tensorflow:Assets written to: /tmp/tmpybb6rhno/assets
/home/polina/test/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-18 23:41:27.086769: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 23:41:27.086822: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 23:41:27.087102: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpybb6rhno
2024-06-18 23:41:27.095430: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 23:41:27.095515: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpybb6rhno
2024-06-18 23:41:27.118967: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 23:41:27.

In [21]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 65.80%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Non-overlap mean F1-score:  0.0

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.0

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Non-overlap mean F1-score:  0.0

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.0

Time for Test dataset:
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Time to make a prediction for a single data point
Mean: 0.014 seconds
Max: 0.028 seconds
Min: 0.012 seconds


Model file name:  time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float_fallback_q.tflite
File size: 0.085 Megabytes


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
